# Introduction


Sentiment analysis is part of the Natural Language Processing (NLP) techniques that consists in extracting emotions related to some raw texts. This is usually used on social media posts and customer reviews in order to automatically understand if some users are positive or negative and why. The goal of this study is to show how sentiment analysis can be performed using python. Here are some of the main libraries we will use:


We will use here some hotel reviews data. Each observation consists in one customer review for one hotel. Each customer review is composed of a textual feedback of the customer's experience at the hotel and an overall rating.

For each textual review, we want to predict if it corresponds to a positive review (the customer is happy) or to a negative one (the customer is not satisfied). 

The challenge here is to be able to predict this information using only the raw textual data from the review. Let's get it started!

### 1. Import the needed libraries
---
> Run the following commands:


pip install pyarabic

pip install nltk

pip install textblob



In [1]:
import sys

!{sys.executable} -m pip install pyarabic
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install TextBlob


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 10.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import pyarabic.araby as araby
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from textblob import TextBlob
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.




> Expand the size of coulmn



In [3]:
# change the seeting of the cells to show the entire content of coulmn 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### 2. Load the Data 
--- 
>It is required to first read the data and ensure that it has ben parsed correctly. The following code block will read and store the .csv file into a data frame that we will clean up accordingly. Then we will perform some data assessment and check for quality issues.


In [4]:
# Mounting Google Drive
from google.colab import drive  # to mount Google Drive to Colab notebook
drive.mount('/content/gdrive')
#path = './gdrive/My Drive/SharedFolder/Data_and_Code/'
#path = './gdrive/My Drive/KSUSentAnalysisWorkshop/SharedFolder/Data_and_Code/'
path = './gdrive/My Drive/SWE486Team2/Data_and_Code/'

Mounted at /content/gdrive


In [5]:
!pwd

/content


In [6]:
data_df=pd.read_csv('/content/gdrive/MyDrive/SWE486Team2 /Data_and_Code/HDataSet.csv') 


###3. Exploratory Data Analysis
> Time to know the data. Time to get insights, and to identify potential problems like missing values, duplications and other issues .

In [7]:
display(data_df.head())
display(data_df.tail())

,reviews,sentiment
0,- أقترح أن تتجنب فندق كوراليا كلوب وأن تنفق أموالك التي تعبت في الحصول عليها في أماكن أرقى في المنطقة. \nلقد كانت إقامتنا فظيعة في هذا المكان خلال أبريل 2011؛ فريق العمل كان فظًا وغير متعاون. الفندق كان يخضع لأعمال التجديد، ونتيجة لذلك خلال ساعات الذروة لا يستطيع المرء الحصول على مقعد لتناول وجبة الإفطار أو العشاء بدون الشعور بالعجلة والضيق، عند طلب مشروب مع وجبة الإفطار أو العشاء كنا ننتهي من الوجبة قبل أن يصل المشروب. خدمة مروعة وأعمال التجديد تؤدي إلى الضوضاء، عطلة غير مريحة، فريق العمل غير أمناء ويقدمون الوعود الكاذبة. أقترح تجنب كوراليا كلوب، وأنفق نقودك التي تعبت في تحصيلها على أماكن أرقى في المنطقة.,Negative
1,"!بشع! المومسات يعشن إلى جواره! \nعند الوصول إلى الفندق فإنه يبدو مختلفًا تمامًا عما هو في الصور. القمامة تنتشر في كل مكان، وحمام السباحة فارغ غير منظم، وكانت المنطقة المستخدمة كحمام للحيوانات الأليفة مليئة بالقمامة والنفايات. لقد كان بجوارنا 4 فتيات قضين في الفندق عدة شهور (بحسب ما قاله المدير)، وقد كن ""لطيفات جدًا"". وقد رأيناهم فيما بعد وهنَّ يتسكعن في الشوارع ويعرضن أنفسهن للبيع، ثم هن قد غادرن الفندق برفقة العديد من الفتيان طوال الليل. وفي صباح اليوم التالي رأينا إحدى هؤلاء الفتيات وهي في طريقها إلى منزلها مرتدية تفس التنورة القصيرة وقد بدت منهكة القوى تمامًا. كان الفندق خاليًا إلى حد كبير إلا من غرفة فيه دخل إليها نحو 8 رجال ثم خرجوا... وبدا أن لو كانت صفقات للمخدرات تجري هناك. ولقد شعرت بالأسى حيال الإدارة لأنها كانت لطيفة للغاية ولكنها بدت وكأنها ساذجة جدًا. (من بلد آخر). كنت على وشك الاتصال بالشرطة...",Negative
2,... \nقمت بزيارة هدا الفندق في ايام عيد الاضحى 2013..\nالفندق غير نظيف..\nطاقم العاملين في الفندق غير لطفاء.. ولم يستقبلونا كما اعتدت في فنادق اخرى..\nالفندق مادي جداً..\nمعاملتهم سيئه.. ( اخص بالدكر طاقم الاستقبال )\nببساطه لولا العائله وشعورنا بالسعادة سوا.. لكانت من اسوأ الرحلات..\n,Negative
3,"""ذا جينجر"" يتراجع \nما الأمر يا ""تاتا جروب""، ما هذا؟!! نزلت في فندق في غينغر بونديشيري (شيناي) قبل عامين واندهشت من المستوى والجودة العالية للفندق - اعتقدت أن هذا حقًا مبدأ تاتا - تغيير اللعبة المعتادة في الفنادق، أي غرف رخيصة لكن مقابل جودة ونظافة عالية ... إلا أني ذهبت إلى دلهي ومررت بتجربة مناقضة لهذه. أصبت بخيبة أمل كبيرة وجعلني ذلك أتردد في الإقامة في فنادق جينجز عندما أزور الهند. ما الأمر يا ""تاتا جروب""، لماذا أقمتم الفندق على فكرة رائعة ثم تركتكم المستوى يتراجع؟ كما أنني أوصيت العائلة بهذا الفندق ""جينجر"" (دلهي) استنادًا إلى تجربتي الرائعة في الإقامة في بونديشيري، وقد أصبتُ بالحرج الشديد عندما ذهبنا إلى هناك!!! إن بونديشيري جينجر فندق رائع رغم ذلك...إنهم فقط بحاجة إلى الحفاظ على نفس المستويات في جميع فنادق جينجر في الهند وسوف يكون الناس على علم بما يتوقعونه من الفنادق - يجب أن يحصل الناس على نفس التجربة الرائعة في كل فندق يذهبون إليه من فنادق السلسلة!! ما الأمر يا ""تاتا جروب""؟! يجب أن تعرفوا ذلك أفضل مني!!",Negative
4,"""رهيب"" \nهذا الفندق لم يشهد أي طبقة من الطلاء أو أي تحديث منذ أن تم افتتاحه، والحمام به بقع من الأتربة على الأرض في كل مكان بالحمام والحوض، وستارة الدش، والمناشف، ومفارش السرير بها ثقوب وأيضًا بقع على الأرض، عمال التنظيف حضروا معظم الأيام لكن لا يوجد دليل على التنظيف في أي مكان، والمناشف تم تغييرها مرة واحدة فقط. الضوضاء كانت مشكلة كبرى، والجدران رقيقة جدًا، ويمكن سماع كل شيء بداية من حركة المرور المتواصلة إلى غرف الجيران وهم يمشون على البلاط وصولًا إلى أصوات الموسيقى، والطريقة الوحيدة التي يمكنك النوم بها هي تكون مخمورًا!! لا توجد غلاية، وأنفقنا مبلغًا كبيرة على هذا الفندق لأننا لم نتحمل الإقامة أو تناول الطعام فيه، ولا ننصح به لأولئك الذين يهتمون بالنظافة على وجه الخصوص. منطقة حمام السباحة محتملة، والمستوى لا بأس به. رخيص وبسيط ويمكن أن يناسب متطلبات الشباب الذين يحتاجون الفندق فقط للنوم.",Negative


,reviews,sentiment
15558,يوم من الراحة في سقارة كونتري كلوب \nفي البداية أود أن أقول أنني لمدة 5 سنوات أعيشها في هذه الدولة لم أندهش من مكان خاص مثل سقارة كونتري كلوب، لقد سمعت عنه كثيرا، ولكن لكوني تجولت في معظم أوقاتي التي عشت فيها في القاهرة وتقريبا أعرف كل مكان في القاهرة، للأمانة لم أعتقد أن هناك مثل هذا المكان السحري جدا والمريح. حديثا قررت أن أذهب وأتحقق منه بعض أن عُرض عليّ ذلك من بعض أصدقائي المصريين القدامى، في البداية اعتقدت أنه سيكون مثل أي مكان آخر أذهب إليه، وما إن دخلت من البوابة وشعرت أن اعتقادي خاطئ. بالتحدث عن هذا المكان، إنه رائع للغاية وكل شيء فيه حمام السباحة الكبير والطعام الشهي جدا بكل أنواعه والخدمات الممتازة وفريق العمل الودود والمحترف ولا أنسى ذكر الممارسات والأنشطة مثل مجموعات الدراجات أو ركوب الخيل في الصحراء التي ترعف كلمة المتعة. بإختصار، كل من يرغب في أوقات رائعة وإقامة عظيمة ومكان للاسترخاء ومكان للقيام ببعض المغامرات الحقيقية، فبالتأكيد سقارة كونتري كلوب هو الخيار المثالي لأنه حقا مكان رائع.\n,Positive
15559,يوم واحد \nقضيت يوم واحد فى هذا الفندق الصغير الرائع . الغرف نظيفة و واسعة . الشرفة واسعة . الطعام ممتاز و متنوع . شرفة المطعم الرئيسى ممتازة ذات اطلالة هادئة .. حمام السباحة الدافئ اتاح لى السباحة فى السادسة صباحا .. فندق رائع\n,Positive
15560,يومين رائعين \nسكنت في هذا الفندق الشهر الماضي كانت الاجواء جميلة والاطلالة جميلة على الشاطئ ببلكونة .. السعر الجيد و الدخول الى السبا مجاني مع سعر الغرفة وتجربه رائعة.. وتشكر ادارة السبا انها اعطتنا خصوصية في السبا لي ولأهلي\n,Positive
15561,مرموق - أفضل عطلة على الإطلاق \nبقينا هنا دون أن نعرف حقا ما يمكن توقعه. كان كل شيء غامضاً بعض الشئ، بعد تجربة فى الصحراء. ولكن هذا المكان رائعاً! الترتيبات تبدو رائعة ، الحيوانات تتجول حول المكان، مما يجعلك تشعر وكأنك في عالم مختلف تماما؛المكان مصمم بطريقة ممتازة وتشعر بالراحة منذ اللحظة التي تصل فيها بل من وقت أن تقود السيارة على الطريق الفرعى الذى يأخذك إلى المكان. الفيلا فاخرة فى الحقيقة ومزودة بمسبح خاص مما جعلنا لا نفكر على الإطلاق فى مغادرة المكان طوال اليوم. لقد استمتعنا بشكل كبير بهذا المسبح وبالإسترخاء حول المسبح بالمناخ الجميل لدولة الإمارات العربية المتحدة بالطبع. شاهدنا بعض التعليقات من قبل ضرر التعرض لأشعة الشمس، ولكن فى الفيلا التي كنا نقيم فيها كان كل شئ على ما يرام. ربما يكون أفضل ما فى المنتجع هو الموظفين ،فهم جميعا حريصون على المساعدة ولطفاء جداً. لم يكن لديهم شيئا مستحيلاً. ففى عدة مرات قاموا بتقديم خدمات لنا حتى قبل طلبنا لها. وكانت إبتسامتهم حاضرة على الدوام، حقاً لقد شعرنا وكأننا فى منزلنا . الطعام كان ممتازاً والتايلاندي على وجه الخصوص كان أفضل ما أكلت! هناك العديد من الأنشطة، من إسطبلات الخيول، وأماكن الطيور الجميلة والعديد من الأنشطة الصحراوية، على الرغم من أننا لم نرى الكثير منها (فالفيلا كانت لطيفة جداً، حيث أننا قضينا كثير من الوقت هناك). يمكنك أيضا قضاء النهار على شاطئ النادى وذهبنا مرة واحدة و قد كانت جيدة جداً، لنادى الصحى به تجهيزات هيدروليكية فائقة! التدليك الذى حصلت عليه كان ممتازاً وجعلنى أشعر براحة كبيرةً. إنه مركز صحى رائع فأنا لا يمكن أن الإنتظار للعودة مرةً أخرى!,Positive
15562,"لا يرقى للنجوم الخمسة \nاطلاله الفندق جميلة جدا والنشاطات الترفيهية متنوعة ولكن احد موظفين الاستقبال ""يوسف"" كان كشر وكرهنا في الفندق..\nخدمات الغرفة جدا ضعيفة لا ترقى للخمس نجوم\nوالنظافة العامة متوسطه\nالمطاعم : سيئة في الترتيب ،الغداء والعشاء بالكاد تحصل شيئ يؤكل لكن الفطور ممتاز\nخدمات الغرفة جدا ضعيفة لا ترقى للخمس نجوم\nتجربه يأسفني انها مش ممكن تتكرر تاني\n",Negative


In [8]:
print("Number of reviews and columns", data_df.shape)
print(data_df.columns)

Number of reviews and columns (15563, 2)
Index(['reviews', 'sentiment'], dtype='object')


In [9]:
data_df.dtypes

reviews      object
sentiment    object
dtype: object

In [10]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15563 entries, 0 to 15562
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   reviews    15563 non-null  object
 1   sentiment  15563 non-null  object
dtypes: object(2)
memory usage: 243.3+ KB


In [11]:
#Summary of the dataset
data_df.describe()

,reviews,sentiment
count,15563,15563
unique,15559,4
top,اجمل فنادق دبي \nالفندق يتميز بطاقم بشري مؤهل ومحب وودود ويتفاني لخدمة العميل . للمره الثانيه انزل بهذا الفندق واجد نفس الحفاوة والترحيب . رغم انه في دبي المنافسه شرسه بين الفنادق في الفخامه والخدمه الا ان رافلز اعتقد انه هو القادر علي جعل اقامتك في دبي مختلفه وذات ذكري جميله جداً .. الغرف واسعه .. الابتسامه هي سمة كل العاملين بالاوتيل .. الاتقان السرعه في الخدمه شي يجعل الرافلز مختلف عن باقي فنادق دبي .. باعتقادي الرافلز اجمل فنادق دبي .\n,Positive
freq,2,10532


### 4. Asses the dataframe
---
We will be assessing both duplicates and null values and based on our observation we will conduct the proper cleaning process
> We will find the duplicates of the reviews column only, since the sentiment column is gaurnteed to have duplicates and they it is simply metadata. The function gets the duplicate rows without the first occurunce. we will then sum and check how much of our data is duplicated.


In [12]:
#Assess null values
print("Number of null values is \n", data_df.isnull().sum())


Number of null values is 
 reviews      0
sentiment    0
dtype: int64


In [13]:
# Issue 1 - Assess Duplicates
duplicate_reviews = data_df[data_df.duplicated(['reviews'], keep='first')]
display(duplicate_reviews.tail())

print(f'{duplicate_reviews.shape[0]} duplicates')

,reviews,sentiment
5144,Comfortable stay \nموقع الفندق مناسب جدا ﻻى سائح حيث انه قريب من المترو واﻻسواق مما يسهل جدا الحركه كما ان استقبال الموظفين رائع والنظافة جيدة جدا ولكن هناك ملحوظة واحدة هه رائحة الفوط غير جيدة بالرغم من نظافتها ولكن غير ذلك ﻻيوجد اي ملحوظة فهو فندق اربعه نجوم ولكن تحصل على خدمة خمس نجوم. ق\n,Positive
6501,اجمل فنادق دبي \nالفندق يتميز بطاقم بشري مؤهل ومحب وودود ويتفاني لخدمة العميل . للمره الثانيه انزل بهذا الفندق واجد نفس الحفاوة والترحيب . رغم انه في دبي المنافسه شرسه بين الفنادق في الفخامه والخدمه الا ان رافلز اعتقد انه هو القادر علي جعل اقامتك في دبي مختلفه وذات ذكري جميله جداً .. الغرف واسعه .. الابتسامه هي سمة كل العاملين بالاوتيل .. الاتقان السرعه في الخدمه شي يجعل الرافلز مختلف عن باقي فنادق دبي .. باعتقادي الرافلز اجمل فنادق دبي .\n,Positive
8609,جيد للغاية!!! \nوصلنا بعد رحلة طويلة ونوم قليل! توجهنا مباشرة لرؤية الفندق وكان أمامنا 14 ساعة في المطار. كانت التكلفة معقولة؛ 50 جنيه استرليني للفرد وكنا نشعر بتعب شديد وكنا سندفع أكثر إذا ما تطلب الأمر ذلك. كانت غرفة الفندق نفسها جميلة بحق، الغرفة والحمام كانا نظيفين. كان الفندق جميل وهادئ لذلك كان من السهل أن ننام على الفور. كان رائعًا أن نجد المياة والشاي مجانًا عند استيقاظنا، كان رائعًا أيضًا أن نستحم مع وجود مناشف نظيفة وجيل وشامبو مجاني. يقع الفندق على بعد دقيقتين من البوابة رقم واحد ولست مضطر حتى إلى مغادرة المطار. سأوصي به بالفعل وشكرًا للفندق على الإقامة الجميلة :),Positive
11785,فندق عائلي ممتاز للعائلة المحافظة \nموقع الفندق ممتاز لوجوده في منطقة ديرة وأمام مول ديرة ستي سنتر وقربه من المترو ووجود مواقف سيارات مجاني حجزت شقة لغير المدخنين لكن للأسف كانت شقتي في الدور السادس المخصص للمدخنين ( بالرغم من إنها نظيفة ) ولكنها سلبية على الفندق النظافة فيه ممتازة النت مجاني لجهازين فقط ولكنه سيء تعامل الموظفين ممتاز وأخص موظف مصري اسمه محمد حيث كان جداً متعاون ومتجاوب عموماً من مميزات الفندق الموقع ووجود الموظف ( محمد المصري )\n,Positive


4 duplicates


### 4. Clean data


In [14]:
# Issue 1 - Removal of duplicates
data_df = data_df.drop_duplicates(subset='reviews', keep="first")
data_df = data_df.reset_index(drop=True)
reviews_copy = data_df.copy()

In [15]:
display(data_df.head(3))
display(data_df.tail(3))
print("Length after removing duplicates", data_df.shape[0])

,reviews,sentiment
0,- أقترح أن تتجنب فندق كوراليا كلوب وأن تنفق أموالك التي تعبت في الحصول عليها في أماكن أرقى في المنطقة. \nلقد كانت إقامتنا فظيعة في هذا المكان خلال أبريل 2011؛ فريق العمل كان فظًا وغير متعاون. الفندق كان يخضع لأعمال التجديد، ونتيجة لذلك خلال ساعات الذروة لا يستطيع المرء الحصول على مقعد لتناول وجبة الإفطار أو العشاء بدون الشعور بالعجلة والضيق، عند طلب مشروب مع وجبة الإفطار أو العشاء كنا ننتهي من الوجبة قبل أن يصل المشروب. خدمة مروعة وأعمال التجديد تؤدي إلى الضوضاء، عطلة غير مريحة، فريق العمل غير أمناء ويقدمون الوعود الكاذبة. أقترح تجنب كوراليا كلوب، وأنفق نقودك التي تعبت في تحصيلها على أماكن أرقى في المنطقة.,Negative
1,"!بشع! المومسات يعشن إلى جواره! \nعند الوصول إلى الفندق فإنه يبدو مختلفًا تمامًا عما هو في الصور. القمامة تنتشر في كل مكان، وحمام السباحة فارغ غير منظم، وكانت المنطقة المستخدمة كحمام للحيوانات الأليفة مليئة بالقمامة والنفايات. لقد كان بجوارنا 4 فتيات قضين في الفندق عدة شهور (بحسب ما قاله المدير)، وقد كن ""لطيفات جدًا"". وقد رأيناهم فيما بعد وهنَّ يتسكعن في الشوارع ويعرضن أنفسهن للبيع، ثم هن قد غادرن الفندق برفقة العديد من الفتيان طوال الليل. وفي صباح اليوم التالي رأينا إحدى هؤلاء الفتيات وهي في طريقها إلى منزلها مرتدية تفس التنورة القصيرة وقد بدت منهكة القوى تمامًا. كان الفندق خاليًا إلى حد كبير إلا من غرفة فيه دخل إليها نحو 8 رجال ثم خرجوا... وبدا أن لو كانت صفقات للمخدرات تجري هناك. ولقد شعرت بالأسى حيال الإدارة لأنها كانت لطيفة للغاية ولكنها بدت وكأنها ساذجة جدًا. (من بلد آخر). كنت على وشك الاتصال بالشرطة...",Negative
2,... \nقمت بزيارة هدا الفندق في ايام عيد الاضحى 2013..\nالفندق غير نظيف..\nطاقم العاملين في الفندق غير لطفاء.. ولم يستقبلونا كما اعتدت في فنادق اخرى..\nالفندق مادي جداً..\nمعاملتهم سيئه.. ( اخص بالدكر طاقم الاستقبال )\nببساطه لولا العائله وشعورنا بالسعادة سوا.. لكانت من اسوأ الرحلات..\n,Negative


,reviews,sentiment
15556,يومين رائعين \nسكنت في هذا الفندق الشهر الماضي كانت الاجواء جميلة والاطلالة جميلة على الشاطئ ببلكونة .. السعر الجيد و الدخول الى السبا مجاني مع سعر الغرفة وتجربه رائعة.. وتشكر ادارة السبا انها اعطتنا خصوصية في السبا لي ولأهلي\n,Positive
15557,مرموق - أفضل عطلة على الإطلاق \nبقينا هنا دون أن نعرف حقا ما يمكن توقعه. كان كل شيء غامضاً بعض الشئ، بعد تجربة فى الصحراء. ولكن هذا المكان رائعاً! الترتيبات تبدو رائعة ، الحيوانات تتجول حول المكان، مما يجعلك تشعر وكأنك في عالم مختلف تماما؛المكان مصمم بطريقة ممتازة وتشعر بالراحة منذ اللحظة التي تصل فيها بل من وقت أن تقود السيارة على الطريق الفرعى الذى يأخذك إلى المكان. الفيلا فاخرة فى الحقيقة ومزودة بمسبح خاص مما جعلنا لا نفكر على الإطلاق فى مغادرة المكان طوال اليوم. لقد استمتعنا بشكل كبير بهذا المسبح وبالإسترخاء حول المسبح بالمناخ الجميل لدولة الإمارات العربية المتحدة بالطبع. شاهدنا بعض التعليقات من قبل ضرر التعرض لأشعة الشمس، ولكن فى الفيلا التي كنا نقيم فيها كان كل شئ على ما يرام. ربما يكون أفضل ما فى المنتجع هو الموظفين ،فهم جميعا حريصون على المساعدة ولطفاء جداً. لم يكن لديهم شيئا مستحيلاً. ففى عدة مرات قاموا بتقديم خدمات لنا حتى قبل طلبنا لها. وكانت إبتسامتهم حاضرة على الدوام، حقاً لقد شعرنا وكأننا فى منزلنا . الطعام كان ممتازاً والتايلاندي على وجه الخصوص كان أفضل ما أكلت! هناك العديد من الأنشطة، من إسطبلات الخيول، وأماكن الطيور الجميلة والعديد من الأنشطة الصحراوية، على الرغم من أننا لم نرى الكثير منها (فالفيلا كانت لطيفة جداً، حيث أننا قضينا كثير من الوقت هناك). يمكنك أيضا قضاء النهار على شاطئ النادى وذهبنا مرة واحدة و قد كانت جيدة جداً، لنادى الصحى به تجهيزات هيدروليكية فائقة! التدليك الذى حصلت عليه كان ممتازاً وجعلنى أشعر براحة كبيرةً. إنه مركز صحى رائع فأنا لا يمكن أن الإنتظار للعودة مرةً أخرى!,Positive
15558,"لا يرقى للنجوم الخمسة \nاطلاله الفندق جميلة جدا والنشاطات الترفيهية متنوعة ولكن احد موظفين الاستقبال ""يوسف"" كان كشر وكرهنا في الفندق..\nخدمات الغرفة جدا ضعيفة لا ترقى للخمس نجوم\nوالنظافة العامة متوسطه\nالمطاعم : سيئة في الترتيب ،الغداء والعشاء بالكاد تحصل شيئ يؤكل لكن الفطور ممتاز\nخدمات الغرفة جدا ضعيفة لا ترقى للخمس نجوم\nتجربه يأسفني انها مش ممكن تتكرر تاني\n",Negative


Length after removing duplicates 15559


In [16]:
# Issue 2 - removal of punctuation
Punct= '''÷×_-“…”!|+~{}’,.؟”:/،_][%^&*()_<>؛'''+ string.punctuation
def removePunctuation(text):
   translator = str.maketrans('', '', Punct)
   text = text.translate(translator)
   text = ' '.join(word for word in text.split())
   return text

In [17]:
# Issue 3 - Removal of unrelated characters 
def remove_unrelated_chars(reviews):
    reviews = re.sub('[_]', ' ',  reviews) 
    reviews = re.sub('#([^\s]+)', r'\\u003c\\1',  reviews) 
    reviews = re.sub('@([^\s]+)', r' ',  reviews) 
    reviews = reviews.replace('\n', ' ')
    reviews = reviews.replace('\t', ' ')
    reviews = reviews.replace('\r', ' ')
    return reviews


In [18]:
# Issue 4 - Removal of arabic stop-word 
ar_stops = set(stopwords.words('arabic'))
stop_words = {"،","","ياعالم","ورحمة","وبركاته","عليكم","السلام","آض","آمينَ","آه","لل","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","مرحباا","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","تحياتي","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","بالعافية","لما","لمّا","فمان","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","اخر","والشفاء","بالهناء","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","ل","لَيْتَ","ل","لَيْسَا","لَيْسَتَا","بتاريخ","لَيْسُوا","لَِسْنَا","فقط","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","5-","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","و","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","فطنت","الكريم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","والله","يكون","يمكن","يوم","ّأيّان"}
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in ar_stops and not w in stop_words and len(w) >= 2])

In [19]:
# Issue 5 - Removal of non-arabic words
def remove_non_arabic_words(reviews):
    #remove English words
     reviews = re.sub(r'\s*[A-Za-z]+\b', ' ' , reviews).rstrip()

    #remove non-Arabic words
     reviews = re.sub(r'[^0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9]+', ' ', reviews)
     return reviews


In [21]:
# Issue 6 - Removal of Emojis
def remove_Emojis(reviews):
    reviews = re.sub('[/\W+/g]', ' ',  reviews)
    return reviews

In [47]:
# Function to clean up and process the reviews - it will be called for every record in reviews column
def clean_review(reviews):
  reviews=removePunctuation(reviews)
  reviews=remove_unrelated_chars(reviews)
  reviews=remove_stop_words(reviews)
  reviews=remove_non_arabic_words(reviews)
  reviews=remove_Emojis(reviews)
  return reviews

In [48]:
# apply the method in all the text and creat another cloumn with the processed text
data_df["reviews"] = data_df['reviews'].apply(lambda x: clean_review(x))

In [49]:
data_df['reviews'] = data_df['reviews'].str.replace('\d+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [50]:
# check the new column
data_df.head()

,reviews,sentiment
0,أقترح تتجنب فندق كوراليا كلوب وأن تنفق أموالك تعبت الحصول أماكن أرقى المنطقة لقد إقامتنا فظيعة المكان فريق العمل فظ وغير متعاون الفندق يخضع لأعمال التجديد ونتيجة لذلك ساعات الذروة يستطيع المرء الحصول مقعد لتناول وجبة الإفطار العشاء بدون الشعور بالعجلة والضيق طلب مشروب وجبة الإفطار العشاء كنا ننتهي الوجبة يصل المشروب خدمة مروعة وأعمال التجديد تؤدي الضوضاء عطلة مريحة فريق العمل أمناء ويقدمون الوعود الكاذبة أقترح تجنب كوراليا كلوب وأنفق نقودك تعبت تحصيلها أماكن أرقى المنطقة,Negative
1,بشع المومسات يعشن جواره الوصول الفندق فإنه يبدو مختلف تمام الصور القمامة تنتشر مكان وحمام السباحة فارغ منظم المنطقة المستخدمة كحمام للحيوانات الأليفة مليئة بالقمامة والنفايات لقد بجوارنا فتيات قضين الفندق شهور بحسب قاله المدير لطيفات جد رأيناهم وهن يتسكعن الشوارع ويعرضن أنفسهن للبيع غادرن الفندق برفقة العديد الفتيان طوال الليل التالي رأينا الفتيات طريقها منزلها مرتدية تفس التنورة القصيرة بدت منهكة القوى تمام الفندق خالي حد كبير غرفة دخل إليها رجال خرجوا وبدا صفقات للمخدرات تجري ولقد شعرت بالأسى حيال الإدارة لأنها لطيفة للغاية ولكنها بدت وكأنها ساذجة جد بلد آخر كنت وشك الاتصال بالشرطة,Negative
2,قمت بزيارة هدا الفندق عيد الاضحى الفندق نظيف طاقم العاملين الفندق لطفاء ولم يستقبلونا اعتدت فنادق الفندق مادي جدا معاملتهم سيئه اخص بالدكر طاقم الاستقبال ببساطه العائله وشعورنا بالسعادة سوا لكانت اسوأ الرحلات,Negative
3,جينجر يتراجع الأمر تاتا جروب نزلت فندق غينغر بونديشيري شيناي عامين واندهشت المستوى والجودة العالية للفندق اعتقدت حق مبدأ تاتا تغيير اللعبة المعتادة الفنادق غرف رخيصة جودة ونظافة عالية أني ذهبت دلهي ومررت بتجربة مناقضة لهذه أصبت بخيبة أمل كبيرة وجعلني أتردد الإقامة فنادق جينجز أزور الهند الأمر تاتا جروب لماذا أقمتم الفندق فكرة رائعة تركتكم المستوى يتراجع أنني أوصيت العائلة بهذا الفندق جينجر دلهي استناد تجربتي الرائعة الإقامة بونديشيري أصبت بالحرج الشديد ذهبنا بونديشيري جينجر فندق رائع رغم ذلكإنهم بحاجة الحفاظ المستويات فنادق جينجر الهند وسوف الناس يتوقعونه الفنادق يجب يحصل الناس التجربة الرائعة فندق يذهبون إليه فنادق السلسلة الأمر تاتا جروب يجب تعرفوا أفضل مني,Negative
4,رهيب الفندق يشهد طبقة الطلاء تحديث افتتاحه والحمام بقع الأتربة الأرض مكان بالحمام والحوض وستارة الدش والمناشف ومفارش السرير ثقوب وأيض بقع الأرض عمال التنظيف حضروا معظم الأيام يوجد دليل التنظيف مكان والمناشف تغييرها مرة واحدة الضوضاء مشكلة كبرى والجدران رقيقة جد ويمكن سماع شيء بداية حركة المرور المتواصلة غرف الجيران وهم يمشون البلاط وصول أصوات الموسيقى والطريقة الوحيدة يمكنك النوم تكون مخمور توجد غلاية وأنفقنا مبلغ كبيرة الفندق لأننا نتحمل الإقامة تناول الطعام ننصح لأولئك يهتمون بالنظافة وجه الخصوص منطقة حمام السباحة محتملة والمستوى بأس رخيص وبسيط ويمكن يناسب متطلبات الشباب يحتاجون الفندق للنوم,Negative


>To clean textual data, we call our custom 'clean_text' function that performs several transformations:


remove the punctuation

remove useless unrelated characters

remove useless stop words

remove emojis 

remove non-arabic words

>Now that we have cleaned our data, we can go ahead and save our dataframe

### 4. Save the Cleaned Data
--- 
>Finally we will save the data back as a .csv file.


In [52]:
cleaned_reviews = data_df.copy()

cleaned_reviews = cleaned_reviews.reset_index(drop=True)

cleaned_reviews.to_csv('final_reviews_cleaned.csv')